# Amini Soil Prediction Challenge

#### Load required packages

In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [150]:
# Load datasets
train_df = pd.read_csv('Train.csv')
train_gap_df = pd.read_csv('Gap_Train.csv')
test_df = pd.read_csv('Test.csv')
test_gap_df = pd.read_csv('Gap_Test.csv')

### Check the shpe of test and train data

In [170]:
train_df.shape, train_gap_df.shape

((7744, 44), (85184, 5))

In [152]:
# Pivot train_gap_df so each PID has one row and each nutrient has its own column
gap_wide = train_gap_df.pivot(index="PID", columns="Nutrient", values="Gap")

# Rename columns to make them clear as gap targets
gap_wide.columns = [f"Gap_{col}" for col in gap_wide.columns]

# Reset index so PID is a column again (not the index)
gap_wide = gap_wide.reset_index()

# Merge gap values into train_df using PID
merged_train_df = train_df.merge(gap_wide, on="PID", how="left")

# Preview the result
merged_train_df.head()


,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,Gap_Ca,Gap_Cu,Gap_Fe,Gap_K,Gap_Mg,Gap_Mn,Gap_N,Gap_P,Gap_S,Gap_Zn
0,site_id_bIEHwl,ID_I5RGjv,70.603761,46.173798,7.75,176,248,920,108,190,...,-19931.6,-8.5016,-218.784,-377.24,-6737.20,-247.80,-3696.0,39.0072,-4.5272,-1.9944
1,site_id_nGvnKc,ID_8jWzJ5,70.590479,46.078924,7.10,181,250,1080,113,191,...,-3575.2,-12.9328,-291.648,-407.04,-706.40,-1242.96,-4156.0,4.4320,-46.9760,-7.4128
2,site_id_nGvnKc,ID_UgzkN8,70.582553,46.048820,6.95,188,250,1109,111,191,...,-5506.8,-3.4208,-223.164,-388.92,-996.48,-189.40,-10120.0,-23.6560,-20.1200,-5.2940
3,site_id_nGvnKc,ID_DLLHM9,70.573267,46.021910,7.83,174,250,1149,112,191,...,-19701.6,-8.9168,-241.624,-542.96,-2120.24,-215.68,-6708.0,-78.1040,-32.1040,-14.1040
4,site_id_7SA9rO,ID_d009mj,70.585330,46.204336,8.07,188,250,869,114,191,...,-20980.4,-8.4658,-197.684,-205.40,-3309.60,-425.74,-2588.4,37.1400,-12.7676,-1.1730


In [153]:
for col in gap_wide.columns:
  print(col)

PID
Gap_B
Gap_Ca
Gap_Cu
Gap_Fe
Gap_K
Gap_Mg
Gap_Mn
Gap_N
Gap_P
Gap_S
Gap_Zn


In [154]:
list(gap_wide.columns)

['PID',
 'Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn']

In [171]:
merged_train_df.shape

(7744, 55)

In [172]:
# Merge the gap data into the train dataset on PID
merged_train_df = train_df.merge(gap_wide, on='PID', how='left')

In [173]:
merged_train_df.head()

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,Gap_Ca,Gap_Cu,Gap_Fe,Gap_K,Gap_Mg,Gap_Mn,Gap_N,Gap_P,Gap_S,Gap_Zn
0,site_id_bIEHwl,ID_I5RGjv,70.603761,46.173798,7.75,176,248,920,108,190,...,-19931.6,-8.5016,-218.784,-377.24,-6737.20,-247.80,-3696.0,39.0072,-4.5272,-1.9944
1,site_id_nGvnKc,ID_8jWzJ5,70.590479,46.078924,7.10,181,250,1080,113,191,...,-3575.2,-12.9328,-291.648,-407.04,-706.40,-1242.96,-4156.0,4.4320,-46.9760,-7.4128
2,site_id_nGvnKc,ID_UgzkN8,70.582553,46.048820,6.95,188,250,1109,111,191,...,-5506.8,-3.4208,-223.164,-388.92,-996.48,-189.40,-10120.0,-23.6560,-20.1200,-5.2940
3,site_id_nGvnKc,ID_DLLHM9,70.573267,46.021910,7.83,174,250,1149,112,191,...,-19701.6,-8.9168,-241.624,-542.96,-2120.24,-215.68,-6708.0,-78.1040,-32.1040,-14.1040
4,site_id_7SA9rO,ID_d009mj,70.585330,46.204336,8.07,188,250,869,114,191,...,-20980.4,-8.4658,-197.684,-205.40,-3309.60,-425.74,-2588.4,37.1400,-12.7676,-1.1730


In [174]:
merged_train_df.shape

(7744, 55)

In [157]:
for col in merged_train_df.columns:
    print(col)

site
PID
lon
lat
pH
alb
bio1
bio12
bio15
bio7
bp
cec20
dows
ecec20
hp20
ls
lstd
lstn
mb1
mb2
mb3
mb7
mdem
para
parv
ph20
slope
snd20
soc20
tim
wp
xhp20
BulkDensity
N
P
K
Ca
Mg
S
Fe
Mn
Zn
Cu
B
Gap_B
Gap_Ca
Gap_Cu
Gap_Fe
Gap_K
Gap_Mg
Gap_Mn
Gap_N
Gap_P
Gap_S
Gap_Zn


In [158]:
merged_train_df.shape

(7744, 55)

#### Feature Selection

In [176]:
# Drop non-feature columns
drop_cols = ['site', 'PID'] + [col for col in train_df.columns if col.startswith('Gap_')] + ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
new_train_data = train_df.drop(columns=drop_cols)


In [177]:
for col in new_train_data.columns:
  print(col)

lon
lat
pH
alb
bio1
bio12
bio15
bio7
bp
cec20
dows
ecec20
hp20
ls
lstd
lstn
mb1
mb2
mb3
mb7
mdem
para
parv
ph20
slope
snd20
soc20
tim
wp
xhp20
BulkDensity


In [178]:
new_train_data.shape

(7744, 31)

In [182]:
missing_counts = new_train_data.isnull().sum()
print(missing_counts)

lon            0
lat            0
pH             0
alb            0
bio1           0
bio12          0
bio15          0
bio7           0
bp             0
cec20          0
dows           0
ecec20         5
hp20           5
ls             0
lstd           0
lstn           0
mb1            0
mb2            0
mb3            0
mb7            0
mdem           0
para           0
parv           0
ph20           0
slope          0
snd20          0
soc20          0
tim            0
wp             0
xhp20          5
BulkDensity    4
dtype: int64


In [183]:
for col in new_train_data.columns:
    n_missing = new_train_data[col].isnull().sum()
    print(f"{col}: {n_missing}")

lon: 0
lat: 0
pH: 0
alb: 0
bio1: 0
bio12: 0
bio15: 0
bio7: 0
bp: 0
cec20: 0
dows: 0
ecec20: 5
hp20: 5
ls: 0
lstd: 0
lstn: 0
mb1: 0
mb2: 0
mb3: 0
mb7: 0
mdem: 0
para: 0
parv: 0
ph20: 0
slope: 0
snd20: 0
soc20: 0
tim: 0
wp: 0
xhp20: 5
BulkDensity: 4


In [184]:
from sklearn.impute import SimpleImputer

# 1) Check missing counts
print("Missing before imputation:")
print(new_train_data.isnull().sum().sort_values(ascending=False).head(10))

# 2) Impute ecec20, hp20, xhp20, BulkDensity with median
imputer = SimpleImputer(strategy='median')
cols_to_impute = ['ecec20', 'hp20', 'xhp20', 'BulkDensity']
new_train_data[cols_to_impute] = imputer.fit_transform(new_train_data[cols_to_impute])

# 3) Verify no more missing
print("\nMissing after imputation:")
print(new_train_data[cols_to_impute].isnull().sum())

Missing before imputation:
hp20           5
ecec20         5
xhp20          5
BulkDensity    4
bio1           0
alb            0
lon            0
bio15          0
bio12          0
bio7           0
dtype: int64

Missing after imputation:
ecec20         0
hp20           0
xhp20          0
BulkDensity    0
dtype: int64


In [186]:
sum_missing = new_train_data.isnull().sum()
print(sum_missing)

lon            0
lat            0
pH             0
alb            0
bio1           0
bio12          0
bio15          0
bio7           0
bp             0
cec20          0
dows           0
ecec20         0
hp20           0
ls             0
lstd           0
lstn           0
mb1            0
mb2            0
mb3            0
mb7            0
mdem           0
para           0
parv           0
ph20           0
slope          0
snd20          0
soc20          0
tim            0
wp             0
xhp20          0
BulkDensity    0
dtype: int64


In [187]:
list(new_train_data.columns)

['lon',
 'lat',
 'pH',
 'alb',
 'bio1',
 'bio12',
 'bio15',
 'bio7',
 'bp',
 'cec20',
 'dows',
 'ecec20',
 'hp20',
 'ls',
 'lstd',
 'lstn',
 'mb1',
 'mb2',
 'mb3',
 'mb7',
 'mdem',
 'para',
 'parv',
 'ph20',
 'slope',
 'snd20',
 'soc20',
 'tim',
 'wp',
 'xhp20',
 'BulkDensity']

In [167]:
gap_cols = [col for col in merged_train_df.columns if col.startswith("Gap_")]
nutrient_cols = ["N", "P", "K", "Ca", "Mg", "S", "Fe", "Mn", "Zn", "Cu", "B"]
drop_for_X = ["site", "PID"] + gap_cols + nutrient_cols

y = merged_train_df[[
  'Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn'
 ]]
X = merged_train_df.drop(columns=drop_for_X)


In [188]:
for col in merged_train_df.columns:
  print(col)

site
PID
lon
lat
pH
alb
bio1
bio12
bio15
bio7
bp
cec20
dows
ecec20
hp20
ls
lstd
lstn
mb1
mb2
mb3
mb7
mdem
para
parv
ph20
slope
snd20
soc20
tim
wp
xhp20
BulkDensity
N
P
K
Ca
Mg
S
Fe
Mn
Zn
Cu
B
Gap_B
Gap_Ca
Gap_Cu
Gap_Fe
Gap_K
Gap_Mg
Gap_Mn
Gap_N
Gap_P
Gap_S
Gap_Zn


In [168]:
print("X columns (features):", X.columns.tolist())
print("X shape:", X.shape)
print("y columns (targets):", y.columns.tolist())
print("y shape:", y.shape)

X columns (features): ['lon', 'lat', 'pH', 'alb', 'bio1', 'bio12', 'bio15', 'bio7', 'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls', 'lstd', 'lstn', 'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'ph20', 'slope', 'snd20', 'soc20', 'tim', 'wp', 'xhp20', 'BulkDensity']
X shape: (7744, 31)
y columns (targets): ['Gap_B', 'Gap_Ca', 'Gap_Cu', 'Gap_Fe', 'Gap_K', 'Gap_Mg', 'Gap_Mn', 'Gap_N', 'Gap_P', 'Gap_S', 'Gap_Zn']
y shape: (7744, 11)


In [148]:
for col in X.columns:
  print(col)

lon
lat
pH
alb
bio1
bio12
bio15
bio7
bp
cec20
dows
ecec20
hp20
ls
lstd
lstn
mb1
mb2
mb3
mb7
mdem
para
parv
ph20
slope
snd20
soc20
tim
wp
xhp20
BulkDensity


## Split the data into Traning, testing and validation sets

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.20, random_state=42
)

print("Training X:", X_train.shape, "Validation X:", X_val.shape)
print("Training y:", y_train.shape, "Validation y:", y_val.shape)

Training X: (6195, 31) Validation X: (1549, 31)
Training y: (6195, 11) Validation y: (1549, 11)


In [62]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Instantiate a base regressor (e.g. RandomForest)
base_rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# Wrap it in MultiOutputRegressor
multi_rf = MultiOutputRegressor(base_rf)

# Fit on the training split
multi_rf.fit(X_train, y_train)

# Predict on validation split
y_pred = multi_rf.predict(X_val)


In [63]:
y_pred.shape

(1549, 11)

In [64]:
from sklearn.metrics import mean_squared_error
import numpy as np

# After you’ve trained and predicted (y_pred is shape [n_val, 11])

for idx, nutrient in enumerate(y_train.columns):
    mse = mean_squared_error(
        y_val.iloc[:, idx],
        y_pred[:, idx]
    )
    rmse = np.sqrt(mse)
    print(f"{nutrient}:   RMSE = {rmse:.3f}")

# Overall RMSE across all nutrient gaps:
overall_mse = mean_squared_error(
    y_val.values.flatten(),
    y_pred.flatten()
)
overall_rmse = np.sqrt(overall_mse)
print(f"\nOverall RMSE (all gaps combined): {overall_rmse:.3f}")


Gap_B:   RMSE = 0.574
Gap_Ca:   RMSE = 3844.033
Gap_Cu:   RMSE = 12.753
Gap_Fe:   RMSE = 104.669
Gap_K:   RMSE = 505.897
Gap_Mg:   RMSE = 852.630
Gap_Mn:   RMSE = 135.136
Gap_N:   RMSE = 1222.015
Gap_P:   RMSE = 115.829
Gap_S:   RMSE = 41.526
Gap_Zn:   RMSE = 6.204

Overall RMSE (all gaps combined): 1253.989


#### Retrain on Full Data

In [80]:
# Pivot train_gap_df so each PID has one row and each nutrient has its own column
gap_wide_test = test_gap_df.pivot(index="PID", columns="Nutrient", values="Required")

# Rename columns to make them clear as gap targets
gap_wide_test.columns = [f"Req_{col}" for col in gap_wide_test.columns]

# Reset index so PID is a column again (not the index)
gap_wide_test = gap_wide_test.reset_index()

# Merge gap values into train_df using PID
merged_test_df = test_df.merge(gap_wide_test, on="PID", how="left")

# Preview the result
merged_test_df.head()

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,Req_Ca,Req_Cu,Req_Fe,Req_K,Req_Mg,Req_Mn,Req_N,Req_P,Req_S,Req_Zn
0,site_id_hgJpkz,ID_NGS9Bx,69.170794,44.522885,6.86,144,256,910,108,186,...,12.0,0.2,0.8,52.0,8.0,0.4,100.0,40.0,12.0,0.4
1,site_id_olmuI5,ID_YdVKXw,68.885265,44.741057,7.08,129,260,851,110,187,...,12.0,0.2,0.8,52.0,8.0,0.4,100.0,40.0,12.0,0.4
2,site_id_PTZdJz,ID_MZAlfE,68.970210,44.675777,6.50,142,259,901,109,187,...,12.0,0.2,0.8,52.0,8.0,0.4,100.0,40.0,12.0,0.4
3,site_id_DOTgr8,ID_GwCCMN,69.068751,44.647707,6.82,142,261,847,109,187,...,12.0,0.2,0.8,52.0,8.0,0.4,100.0,40.0,12.0,0.4
4,site_id_1rQNvy,ID_K8sowf,68.990002,44.577607,6.52,145,253,1109,110,186,...,12.0,0.2,0.8,52.0,8.0,0.4,100.0,40.0,12.0,0.4


In [118]:
# Drop non-feature columns
drop_cols = [
  'PID',
  'site',
  'Req_B',
  'Req_Ca',
  'Req_Cu',
  'Req_Fe',
  'Req_K',
  'Req_Mg',
  'Req_Mn',
  'Req_N',
  'Req_P',
  'Req_S',
  'Req_Zn'
  ]
new_test_data = merged_test_df.drop(columns=drop_cols)

In [119]:
new_test_data.shape

(2418, 31)

In [105]:
for col in new_test_data.columns:
  print(col)

lon
lat
pH
alb
bio1
bio12
bio15
bio7
bp
cec20
dows
ecec20
hp20
ls
lstd
lstn
mb1
mb2
mb3
mb7
mdem
para
parv
ph20
slope
snd20
soc20
tim
wp
xhp20
BulkDensity


In [131]:
X = merged_test_df
y = merged_test_df[['Req_B',
  'Req_Ca',
  'Req_Cu',
  'Req_Fe',
  'Req_K',
  'Req_Mg',
  'Req_Mn',
  'Req_N',
  'Req_P',
  'Req_S',
  'Req_Zn']]

In [132]:
X.shape, y.shape

((2418, 44), (2418, 11))

In [127]:
preds_full = multi_rf_full.predict(X)
preds_full.shape

(7744, 11)

In [129]:
# Build a long‐format DataFrame of predictions
all_preds = []
for idx, nutrient in enumerate(y.columns):
    df_sub = pd.DataFrame({
        'PID': merged_test_df['PID'],
        'Nutrient': nutrient.replace('Req_', ''),
        'Gap': preds_full[:, idx]
    })
    all_preds.append(df_sub)

SampleSubmission = pd.concat(all_preds, axis=0)
SampleSubmission['PID'] = SampleSubmission['PID'].astype(str) + '_' + SampleSubmission['Nutrient']
SampleSubmission = SampleSubmission[['PID', 'Gap']]
SampleSubmission.to_csv('SampleSubmission.csv', index=False)
print("✅ SampleSubmission.csv created (shape:", SampleSubmission.shape, ")")


ValueError: array length 7744 does not match index length 2418

In [130]:
merged_test_df.shape

(2418, 44)

In [122]:
sub_data = pd.read_csv('SampleSubmission.csv')

In [123]:
for col in sub_data.columns:
  print(col)

PID
Gap


In [124]:
sub_data.shape

(26598, 2)